#### DEFINIR CONSTANTES

In [82]:
#----------- Arquivos e drivers ----------------
GZIP_INPUT_FILE = './data/votacao.parquet.gzip'
JDBC_DRIVER_PATH = "./driver/postgresql-42.7.1.jar"

#----------- Configurações do banco de dados ------------
hostname_or_ip = "localhost"
port = "5439"
db = "metabase"
user = "postgres"
password = "postgres"
db_url = "jdbc:postgresql://" + hostname_or_ip + ":" + port + "/" + db
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver"
}

#### CRIANDO SEÇÃO SPARK

In [83]:
from pyspark.sql import SparkSession
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path ${JDBC_DRIVER_PATH} pyspark-shell'
spark = SparkSession.builder.appName('spark').config("spark.driver.extraClassPath", JDBC_DRIVER_PATH).getOrCreate()

#### LER PARQUET FILE

In [84]:
df = spark.read.parquet(GZIP_INPUT_FILE)
df.show()

+--------+--------------------+--------------+-----+--------------------+-------+-----------------+------------+--------------------+--------------------+-----------------------+----------+-------------+--------------------+---------------+-----------------------+-------------------+-----------------+------------------------+-------------------------+----------------+
|NR_TURNO|          DS_ELEICAO|TP_ABRANGENCIA|SG_UF|        NM_MUNICIPIO|NR_ZONA|         DS_CARGO|NR_CANDIDATO|        NM_CANDIDATO|   NM_URNA_CANDIDATO|DS_SITUACAO_CANDIDATURA|NR_PARTIDO|   SG_PARTIDO|          NM_PARTIDO|   NM_COLIGACAO|DS_COMPOSICAO_COLIGACAO|ST_VOTO_EM_TRANSITO|QT_VOTOS_NOMINAIS|NM_TIPO_DESTINACAO_VOTOS|QT_VOTOS_NOMINAIS_VALIDOS|DS_SIT_TOT_TURNO|
+--------+--------------------+--------------+-----+--------------------+-------+-----------------+------------+--------------------+--------------------+-----------------------+----------+-------------+--------------------+---------------+------------------

#### TRANSFORMAR PARQUET PARA STAR SCHEMA

In [85]:
from utils.transform_spark_df_to_star_schema import transform_spark_dataframe_into_star_schema

star_schema = transform_spark_dataframe_into_star_schema(
    df,
    colunas_fato=["QT_VOTOS_NOMINAIS_VALIDOS", "QT_VOTOS_NOMINAIS"],
    tabela_fato_nome="tabela_fato",
    mapping_colunas_dimensao={
        'dim_municipio': ["SG_UF", "NM_MUNICIPIO"],
        'dim_cargo': ["DS_CARGO"],
        'dim_ds_eleicao':["DS_ELEICAO"],
        'dim_partido':["SG_PARTIDO","NM_PARTIDO", "NR_PARTIDO"],
        'dim_candidato':["NM_CANDIDATO", "NR_CANDIDATO","NM_URNA_CANDIDATO"],
        'dim_turno':["NR_TURNO"],
        'dim_tp_agrangencia':["TP_ABRANGENCIA"],
        'dim_zona':["NR_ZONA"],
        'dim_situacao_candidatura':["DS_SITUACAO_CANDIDATURA"],
        'dim_coligacao':["NM_COLIGACAO", "DS_COMPOSICAO_COLIGACAO"],
        'dim_voto_transito':["ST_VOTO_EM_TRANSITO"],
        'dim_destinacao_votos':["NM_TIPO_DESTINACAO_VOTOS"],
        'dim_totalizacao_candidato':["DS_SIT_TOT_TURNO"]
    },   
)

#### VISUALIZANDO DENTRO DAS TABELAS

In [86]:
# Exemplo de como acessar as tabelas:
tabela_fato_nome, df_fato = star_schema[0]
# Visualize o conteúdo da tabela fato
print(f"Conteúdo da Tabela Fato ({tabela_fato_nome}):")
df_fato.show(truncate=False)
# Exiba o esquema da tabela fato
print("Esquema da Tabela Fato:")
df_fato.printSchema()
# Exemplo de como acessar as tabelas de dimensão:
df_dimensao1_nome, df_dimensao1 = star_schema[1]
# Visualize o conteúdo da dimensão 1
print(f"Conteúdo da Dimensão 1 ({df_dimensao1_nome}):")
df_dimensao1.show(truncate=False)
# Exiba o esquema da dimensão 1
print("Esquema da Dimensão 1:")
df_dimensao1.printSchema()
# Exemplo de como acessar as tabelas de dimensão:
df_dimensao1_nome, df_dimensao1 = star_schema[4]
# Visualize o conteúdo da dimensão 1
print(f"Conteúdo da Dimensão 1 ({df_dimensao1_nome}):")
df_dimensao1.show(truncate=False)
# Exiba o esquema da dimensão 1
print("Esquema da Dimensão 1:")
df_dimensao1.printSchema()

Conteúdo da Tabela Fato (tabela_fato):


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\034033021651\Documents\MetaStar\.venv\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\034033021651\Documents\MetaStar\.venv\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

#### INSERIR MODELO ESTRELA NO BANCO DE DADOS

In [87]:
for item in star_schema:
    table_name,dataframe = item
    print(f"Writing {table_name} to DW")
    dataframe.write.jdbc(url=db_url, table=table_name, mode="overwrite", properties=properties)

Writing tabela_fato to DW


Py4JJavaError: An error occurred while calling o3159.jdbc.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.net.URLClassLoader.findClass(Unknown Source)
	at java.lang.ClassLoader.loadClass(Unknown Source)
	at java.lang.ClassLoader.loadClass(Unknown Source)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:246)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:250)
	at org.apache.spark.sql.execution.datasource